In [1]:
# baseline cnn model for digit
import numpy as np
#import import_ipynb
from loader import Loader
from sklearn.model_selection import KFold
from keras.utils import to_categorical
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.layers import BatchNormalization
import random as r
import matplotlib.pyplot as plt
from keras.preprocessing.image import img_to_array, load_img
from matplotlib import pyplot
import scipy.stats as stats
import os

In [2]:
def dataset1(intensity,dataset):
# Dividing the data into train and test data
    #clear = lambda: os.system('clear')
    #clear()
    
    
    noise = intensity
    print('The noise intensity is')
    print(noise)
    
    trainX,trainY =  dataset.getWholeTrainSet(pca=False) 
    trainXwithnoise, trainYwithnoise = dataset.getNoisySet(intensity=noise, set="train", flat=False, truncate=True)
    
    testX, testY = dataset.getWholeTestSet()
    
    trainX = np.array(trainX)
    trainXwithnoise = np.array(trainXwithnoise)
    trainY = np.array(trainY)
    testX = np.array(testX)
    testY = np.array(testY)
   
    trainXwithnoise = trainXwithnoise.reshape((trainXwithnoise.shape[0], 16,15, 1))
    testX = testX.reshape((testX.shape[0], 16,15, 1))
    trainX = trainX.reshape((trainX.shape[0], 16,15, 1))
    # the labels are one hot encoded
    testYintegers = testY
    testY = to_categorical(testY)
    trainY = to_categorical(trainY)
    return trainX,trainXwithnoise, trainY, testX, testY,testYintegers

In [3]:
#dataset11  = Loader()
#trainX,trainXwithnoise, trainY, testX, testY,testYintegers = dataset1(1,dataset11)
#trainXwithnoise1,train_norm1, test_norm1= normalise_data(trainXwithnoise,trainX,testX) 
#print('printing the k fold accuracies with noisy training data')
#accuracies, histories = train_modelkfold(train_norm1,trainXwithnoise1, trainY)  
    

In [4]:
def getdataset1(intensity,dataset):
    
    trainX,trainXwithnoise, trainY, testX, testY,testYintegers = dataset1(intensity,dataset)
    return trainX,trainXwithnoise, trainY, testX, testY,testYintegers

In [5]:
def model_definition():
    # tune model parameters here
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(16,15, 1)))
    #model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Flatten())
    
    
    
    # Flatten and apply drop out or apply drop out after Conv2D it is essentially the same
    model.add(Dropout(0.50))
    model.add(Dense(240, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    
    
    model.add(Dropout(0.20))
    model.add(Dense(120, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    
    model.add(Dropout(0.20))
    model.add(Dense(50, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    
    
    
    
#    model.add(Dropout(0.20))
    model.add(Dense(10, activation='softmax'))
    #opt = SGD(lr=0.01)
    opt = SGD(lr=0.03, momentum=0.9)
    #opt = 'adam'
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [6]:
def train_modelkfold(mtrainX,mtrainXwithnoise, mtrainY, folds=10):
    accuracies,histories = list(), list()

    kfold = KFold(folds, shuffle=True,random_state=3)      
    for training_indices, testing_indices in kfold.split(mtrainX):
        model = model_definition()
        trainX, trainY, testX, testY = mtrainXwithnoise[training_indices], mtrainY[training_indices], mtrainX[testing_indices], mtrainY[testing_indices]
        # so this model.fit reflects the model.fit for only the kfold 
        #model.fit(trainX, trainY, epochs=120, batch_size=10, validation_data=(testX, testY), verbose=0)
        history_kfold = model.fit(trainX, trainY, epochs=200, batch_size=32, validation_data=(testX, testY), verbose=0)
        
        # returning the validation accuracies of every k fold validation set
        _, accuraccy = model.evaluate(testX, testY, verbose=0)
        print('%.3f' % (accuraccy * 100.0))
        # stores accuracies and test data on last kth model
        accuracies.append(accuraccy)
        histories.append(history_kfold)
        # implement a list for storing all the k fold models. If you wish
    return accuracies,histories

In [7]:
# function to print the accuracies of the k fold iterations
def fitmodel(trainX, trainY,testX, testY,i,intensity):   
    
    #once the model has been decided on from k fold
    model = model_definition()
    history_training = model.fit(trainX, trainY,epochs=200, verbose=0)
    # save model
    name = "final_model.h5" + str(i) + str(intensity)
    model.save(name)
    # load the model
    model = load_model(name)
    return model,history_training

In [8]:
def modelpredictions(modellist,testX,testY):
    
    
    correctpernoise = []
    for model in modellist:
        correctaveragepermodel = 0.0
        correct = 0.0
        preds = model.predict(testX)
        preds = np.argmax(model.predict(testX), axis=-1)
        #preds = to_categorical(preds)
        #print(preds)
        
        for i in range(len(testY)):
            if (preds[i] == testY[i]): # Change
                correct = correct+1
                
        correctaveragepermodel = correct/len(testY)
        correctpernoise.append(correctaveragepermodel)
        #return list
    return correctpernoise

In [9]:
def results():
    modellist = []
    testaccuracypernoise = []
    testerrorpernoise = []
    numofmodels = 10
    #intensities = [.05, .1, .15, .20, .25, .30, .35, .40, .45, .50, .55, .6, .65, .7, .75, .8, .85, .9, .95, 1]
    intensities = [.1,.20,.30,.40,.50]
    #datasetlist = [Loader() for i in range(2)]
    dataset = Loader()
    trainingaccuracypernoiseintensity = []
    traininglosspernoiseintensity = []
    histories = list()
    
    for j in range(len(intensities)):
        modellist = []
        trainX,trainXwithnoise, trainY, testX, testY,testYintegers = dataset1(intensities[j],dataset)
       # trainXwithnoise1,train_norm1, test_norm1= normalise_data(trainXwithnoise,trainX,testX) 
        print('printing the k fold accuracies with noisy training data')
        accuracies, histories = train_modelkfold(trainX,trainXwithnoise, trainY)
        print('sum of k-fold accuracies')
        sum(accuracies)
        #accuracies, histories = train_modelkfold(trainX,trainXwithnoise, trainY)
        trainiaccuracy = 0.0
        finalresultpernoise = 0
        errorresultpernoise=0.0
        trainloss = 0.0
        # making 5 models
        for i in range(numofmodels):
            model,history_training = fitmodel(trainXwithnoise, trainY,testX, testY,i,intensities[j])
            histories.append(history_training)
            modellist.append(model)
            print('In training')
            trainiaccuracy = trainiaccuracy + (sum(histories[i].history['accuracy'])/200)
            
        trainiaccuracy = trainiaccuracy/numofmodels
        trainloss = 1-trainiaccuracy
        traininglosspernoiseintensity.append(trainloss)
        trainingaccuracypernoiseintensity.append(trainiaccuracy)       
        #correct = 0    
        print('In training')
        correctpermodel = modelpredictions(modellist,testX,testYintegers)
        
        
        finalresultpernoise = (sum(correctpermodel)/numofmodels)*100
        errorresultpernoise = 100-finalresultpernoise
        testerrorpernoise.append(errorresultpernoise)
        testaccuracypernoise.append(finalresultpernoise)

        %reset -f in 
    return testaccuracypernoise,testerrorpernoise,trainingaccuracypernoiseintensity
    #predslist.append(preds)

In [10]:
testaccuracypernoise,testerrorpernoise,trainingaccuracypernoiseintensity = results()

The noise intensity is
0.1
printing the k fold accuracies with noisy training data
96.000
98.000
94.000
98.000
99.000
98.000
98.000
98.000
98.000
99.000
sum of k-fold accuracies
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: final_model.h500.1/assets
In training
INFO:tensorflow:Assets written to: final_model.h510.1/assets
In training
INFO:tensorflow:Assets written to: final_model.h520.1/assets
In training
INFO:tensorflow:Assets written to: final_model.h530.1/assets
In training
INFO:tensorflow:Assets written to: final_model.h540.1/assets
In training
INFO:tensorflow:Assets written to: final_model.h550.1/assets
In training
INFO:tensorflow:Assets written to: final_model.h560.1/assets
In training
INFO:tensorflow:Assets written to: final_model.h570.1/assets
In training
INFO

In [11]:
testaccuracypernoise

[97.99000000000001, 98.10000000000001, 98.03999999999998, 98.04, 97.83]

In [12]:
testerrorpernoise

[2.009999999999991,
 1.8999999999999915,
 1.9600000000000222,
 1.9599999999999937,
 2.1700000000000017]

In [13]:
trainingaccuracypernoiseintensity

[0.966148333221674,
 0.969826109856367,
 0.9635166661143302,
 0.9666983323693277,
 0.9669233332276344]

In [14]:
1-trainingaccuracypernoiseintensity

TypeError: unsupported operand type(s) for -: 'int' and 'list'